# Tables VIII and IX: Rate Coefficients and Energy Depletion Timelines

This notebook reproduces the values for Tables VIII and IX of
*AI Hastens Limits to Exponential Growth*.

**Table VIII** — Calculated Rate Coefficients $k_1$–$k_5$ for five growth rates $r$.

**Table IX** — Energy Depletion Timelines $t = k/r$ (yr) for a grid of $k$ and $r$ values.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.precision', 1)

## Physical constants and baseline values

In [ ]:
# Physical constants
sigma_B    = 5.670e-8      # Stefan-Boltzmann constant, W/m^2/K^4
s_per_year = 365*24*60*60  # s/yr
J_per_EJ   = 1e18          # J/EJ
m_per_km   = 1e3           # m/km
Tboil      = 373           # K, boiling point of water

# Astronomical parameters
L_W  = 3.828e26  # Solar luminosity, W
Xo   = 150e6     # Earth-Sun distance, km
Xe   = 6.37e3    # Earth radius, km

# Energy resources and demand
R0 = 15.8e12  # Total non-renewable resources (dominated by deuterium), EJ
D0 = 633      # Baseline demand, EJ/yr

# Stellar density
Vinv = 0.004  # stars/ly^3 (local galactic neighbourhood)

# Albedo
albedo = 0.31  # Dmnl

# Derived quantities
L     = L_W * s_per_year / J_per_EJ   # Solar luminosity, EJ/yr
alpha = Xe**2 / (4 * Xo**2)           # Geometric flux fraction, Dmnl

# Earth surface area and equilibrium temperature T0
area_earth = 4 * np.pi * (Xe * m_per_km)**2   # m^2
Pa   = L_W * alpha * (1 - albedo)              # Absorbed solar power, W
T0   = (Pa / (sigma_B * area_earth))**0.25     # Equilibrium surface temp, K

print(f'L     = {L:.3e} EJ/yr')
print(f'alpha = {alpha:.3e}')
print(f'T0    = {T0:.1f} K  (equilibrium surface temperature)')

## Table VIII — Rate Coefficients $k_1$–$k_5$

$$
\begin{align}
k_1(r) &= \ln\!\left[\frac{R(0)}{D(0)/r} + 1\right] \\
k_2     &= \ln\!\left[\frac{L\,\alpha\,(1-a)}{D(0)}\left(\left[\frac{T_2}{T_0}\right]^4-1\right)\right] \\
k_3     &= \ln\!\left[\frac{L\,\alpha}{D(0)}\right] \\
k_4     &= \ln\!\left[\frac{L}{D(0)}\right] \\
k_5(r)  &= \ln\!\left[\frac{L}{D(0)}\cdot\frac{4\pi/3}{V}\cdot\left(\frac{c}{r/3}\right)^3\right]
\end{align}
$$

In [ ]:
r_vals = [0.010, 0.015, 0.030, 0.10, 0.15]  # 1/yr

# Constant coefficients (independent of r)
k2 = np.log(L * alpha * (1 - albedo) / D0 * ((Tboil / T0)**4 - 1))
k3 = np.log(L * alpha / D0)
k4 = np.log(L / D0)

rows = []
for r in r_vals:
    k1 = np.log(R0 / (D0 / r) + 1)  # = ln(R0*r/D0 + 1); dimensionally correct
    Asimov = 4 * np.pi / 3 * Vinv * (3 / r)**3
    k5 = np.log(L / D0 * Asimov)
    rows.append({'r (1/yr)': r, 'k1 Non-Renew.': k1, 'k2 Kelvin': k2,
                 'k3 Renew.': k3, 'k4 Dyson': k4, 'k5 Stellar': k5})

df8 = pd.DataFrame(rows).set_index('r (1/yr)')
print('Table VIII — Calculated Rate Coefficients')
df8.round(1)

## Table IX — Energy Depletion Timelines (yr)

$$t = k / r$$

In [ ]:
k_vals = [10, 20, 30, 40, 50]

col_r = np.array(r_vals).reshape(-1, 1)
row_k = np.array(k_vals).reshape(1, -1)
t_grid = row_k / col_r

df9 = pd.DataFrame(data=t_grid,
                   index=[str(r) for r in r_vals],
                   columns=[f'k={k}' for k in k_vals])
df9.index.name = 'r (1/yr)'

print('Table IX — Energy Depletion Timelines (yr)')
df9.round(0).astype(int)